In [55]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import json
import re

In [56]:
# #############
#   load data
# #############
path_to_open = "..\\data\\_properties_data.json"

data = None
with open(path_to_open, "r", encoding="utf-8") as file:
    data = json.load(file)

df = pd.DataFrame.from_dict(data, orient='index')

df.head()

,URL,Status,transactionType,transactionSubtype,price,type,subtype,country,region,province,...,toiletCount,parkingCountIndoor,parkingCountOutdoor,parkingCountClosedBox,primaryEnergyConsumptionPerSqm,epcScore,hasDoubleGlazing,saleType,creationDate,lastModificationDate
10616910,https://www.immoweb.be/en/classified/10616910,200,FOR_SALE,BUY_REGULAR,399000.0,APARTMENT,APARTMENT,Belgium,Flanders,Limburg,...,1.0,1.0,1.0,None,142.0,B,True,None,2023-06-07T07:39:57.000+0000,2023-06-07T07:39:59.080+0000
10616880,https://www.immoweb.be/en/classified/10616880,200,FOR_SALE,BUY_REGULAR,218000.0,APARTMENT,APARTMENT,Belgium,Flanders,Antwerp,...,1.0,NaN,NaN,None,92.0,A,True,None,2023-06-07T07:22:33.000+0000,2023-07-05T08:30:20.607+0000
10616933,https://www.immoweb.be/en/classified/10616933,200,FOR_SALE,BUY_REGULAR,95000.0,APARTMENT,APARTMENT,Belgium,Flanders,West Flanders,...,1.0,NaN,NaN,None,521.0,F,None,None,2023-06-07T07:52:35.000+0000,2023-07-05T08:28:15.563+0000
10616917,https://www.immoweb.be/en/classified/10616917,200,FOR_SALE,BUY_REGULAR,169000.0,APARTMENT,APARTMENT,Belgium,Flanders,West Flanders,...,NaN,NaN,NaN,None,391.0,D,True,None,2023-06-07T07:47:23.000+0000,2023-07-05T08:19:08.357+0000
10616913,https://www.immoweb.be/en/classified/10616913,200,FOR_SALE,BUY_REGULAR,359000.0,APARTMENT,APARTMENT,Belgium,Flanders,Antwerp,...,NaN,NaN,NaN,None,140.0,B,False,None,2023-06-07T07:44:08.000+0000,2023-06-07T07:44:09.170+0000


In [16]:
# ######################
#   fix locality names
# ######################

In [42]:
# load new locality data
ref_postalCode_locality_data = pd.read_csv("..\data\_postal_code_data.csv")
ref_postalCode_locality_data.head()

,postalCode,city,municipality,latitude,longitude,cityAltName
0,1000,Brussel,Brussel,50.822285,4.381571,Bruxelles
1,1030,Schaarbeek,Schaarbeek,50.822285,4.381571,Schaerbeek
2,1040,Etterbeek,Etterbeek,50.822285,4.381571,NaN
3,1050,Ixelles,Ixelles,50.822285,4.381571,Elsene
4,1060,Saint-Gilles,Saint-Gilles,50.822285,4.381571,Sint-Gillis


In [36]:
df[["postalCode", "locality", "latitude", "longitude"]].head()

,postalCode,locality,latitude,longitude
10616910,3550,Heusden,51.019022,5.307467
10616880,2400,Mol,51.189970,5.116175
10616933,8800,Roeselare,50.941540,3.122149
10616917,8660,De Panne,51.097924,2.592177
10616913,2050,Antwerpen,51.232721,4.378834


In [57]:
"""df["city"] = None
df["municipality"] = None

for index, row in df.iterrows():
    pcode = row["postalCode"]
    locality = row["locality"]
    pcode_data = ref_postalCode_locality_data.query('postalCode == @pcode')
    if len(pcode_data) == 1:
        print("debug " + pcode)
        df.loc[index, "city"] = "test"
        df.loc[index, "municipality"] = "test"
    else:
        #for index, row in pcode_data.iterrows():
        pass

df[["postalCode", "locality", "city", "municipality"]].head(10)
"""

'df["city"] = None\ndf["municipality"] = None\n\nfor index, row in df.iterrows():\n    pcode = row["postalCode"]\n    locality = row["locality"]\n    pcode_data = ref_postalCode_locality_data.query(\'postalCode == @pcode\')\n    if len(pcode_data) == 1:\n        print("debug " + pcode)\n        df.loc[index, "city"] = "test"\n        df.loc[index, "municipality"] = "test"\n    else:\n        #for index, row in pcode_data.iterrows():\n        pass\n\ndf[["postalCode", "locality", "city", "municipality"]].head(10)\n'

In [15]:
# #####################
#   add locality type
# #####################
def addLocalityType(postalCode):
    localityType = 0
    # X000 > 1
    if re.match(r"^[1-9][0]{3}$", postalCode):
        localityType = 1
    # X0X0 > 2
    elif re.match(r"^([1-9][0]){2}$", postalCode):
        localityType = 2
    # X0XX > 3
    elif re.match(r"^[1-9][0][1-9]{2}$", postalCode):
        localityType = 3
    # XX00 > 4
    elif re.match(r"^[1-9]{2}[0]{2}$", postalCode):
        localityType = 4
    # XXX0 > 5
    elif re.match(r"^[1-9]{3}[0]$", postalCode):
        localityType = 5
    # XXXX > 6
    elif re.match(r"^[1-9]{4}$", postalCode):
        localityType = 5
    return localityType

#df["localityType"] = df["postalCode"].apply(addLocalityType)
#df[["postalCode", "localityType"]].head(10)


In [58]:
# #################
#   merge address
# #################
df["address"] = df["street"] + " " + df["number"] #+ ", " + df["box"]
df[["street", "number", "box", "address"]].head()

,street,number,box,address
10616910,Korenhof,14,11,Korenhof 14
10616880,Corbiestraat,98,6,Corbiestraat 98
10616933,Meensesteenweg,30,2B,Meensesteenweg 30
10616917,Kasteelstraat,9,None,Kasteelstraat 9
10616913,August Vermeylenlaan,3,37,August Vermeylenlaan 3


In [59]:
# ###################
#   price per meter  
# ###################
df["pricePerMeter"] = round(df["price"] / df["netHabitableSurface"])
df[["price", "netHabitableSurface", "pricePerMeter"]].head()

,price,netHabitableSurface,pricePerMeter
10616910,399000.0,125.0,3192.0
10616880,218000.0,85.0,2565.0
10616933,95000.0,41.0,2317.0
10616917,169000.0,NaN,NaN
10616913,359000.0,110.0,3264.0


In [60]:
# ################################################
#   apply new order and remove redundant columns
# ################################################
columns_to_take = [
    "type", "subtype", 
    "region", "province", "district", "locality", "postalCode", "address", "latitude", "longitude", 
    "condition", "constructionYear", "price", "pricePerMeter", "bedroomCount", "netHabitableSurface", 
    "floor", "floorCount", "hasLift", "facadeCount", 
    "hasGarden", "gardenSurface", "hasTerrace", "terraceSurface", "land", 
    "kitchen", "bathroomCount", "showerRoomCount", "toiletCount", 
    "fireplaceExists", "hasSwimmingPool", "hasAirConditioning", 
    "parkingCountIndoor", "parkingCountOutdoor", "parkingCountClosedBox", 
    "primaryEnergyConsumptionPerSqm", "epcScore", "hasDoubleGlazing", 
    "saleType" 
    ]
df_reduced = df[columns_to_take]
df_reduced.head()


,type,subtype,region,province,district,locality,postalCode,address,latitude,longitude,...,fireplaceExists,hasSwimmingPool,hasAirConditioning,parkingCountIndoor,parkingCountOutdoor,parkingCountClosedBox,primaryEnergyConsumptionPerSqm,epcScore,hasDoubleGlazing,saleType
10616910,APARTMENT,APARTMENT,Flanders,Limburg,Hasselt,Heusden,3550,Korenhof 14,51.019022,5.307467,...,False,False,False,1.0,1.0,None,142.0,B,True,None
10616880,APARTMENT,APARTMENT,Flanders,Antwerp,Turnhout,Mol,2400,Corbiestraat 98,51.189970,5.116175,...,False,False,None,NaN,NaN,None,92.0,A,True,None
10616933,APARTMENT,APARTMENT,Flanders,West Flanders,Roeselare,Roeselare,8800,Meensesteenweg 30,50.941540,3.122149,...,False,False,None,NaN,NaN,None,521.0,F,None,None
10616917,APARTMENT,APARTMENT,Flanders,West Flanders,Veurne,De Panne,8660,Kasteelstraat 9,51.097924,2.592177,...,False,None,None,NaN,NaN,None,391.0,D,True,None
10616913,APARTMENT,APARTMENT,Flanders,Antwerp,Antwerp,Antwerpen,2050,August Vermeylenlaan 3,51.232721,4.378834,...,False,False,False,NaN,NaN,None,140.0,B,False,None


In [10]:
# ################
#   save new csv
# ################
path_to_save = "..\\data\\01_modifyData_properties_data.csv"
df_reduced.to_csv(path_to_save, index=True, index_label="ID")
